In [1]:
import plotly.graph_objects as go
from scipy.stats import norm
import math
import numpy as np

# Options

**Call** Option = Option to **Buy**

**Put** Option = Option to **Sell**

* $S$ = Current stock price
* $X$ = Exercise price (Option value if exercised today $= max(0, CurrentS - x$)
* $u$ = Upwards movement
* $d$ = Downwards movement
* $r_f$ = Risk-free rate

In calls: The portfolio is composed of one long stocks and m call options

In puts: The portfolio is composed of m long stocks and one long put option

Types:
* **European**: Can only be exercised at expiration date
* **American**: Can be exercised at any time before expiration


Phrases:
* **In-the-money call**: Call where currently strike X < S
* **Out-of-the-money call**: Call where currently strike X > S
* **At-the-money call**: Call where currently strike X ≈ S

## Binomial Tree Option Pricing

**Assumptions**
* Trading occurs only at dicrete times
* Stock price follows multiplicative binomial process (Two possible states of the world in each step)

**Basic Idea:** Construct risk-free hedge portfolio, composed of
* one share of stock
* m shares of call option on stock

For the portfolio to be risk-free, end-of-period payoffs must be equal in each state
* $uS -mc_u = dS - mc_d$
m is number of options needed for hedge (hedge ratio)
* $m = \frac{uS - dS}{c_u - c_d} = \frac{S(u - d)}{c_u - c_d}$
Because hedge portfolio is constructed to be risk-less it must hold
* $(1+ r_f )(S - mc) = uS - mc_u$
* $\Leftrightarrow c = \frac{S[(1+r_f)-u]+ mc_u}{m(1+r_f)}$
Substituting the hedge ratio m into this pricing equation yields
* $c = \left[c_u\left(\frac{(1+r_f)-d}{u-d}\right)+c_d\left(\frac{u-(1+r_f)}{u-d}\right)\right]\div(1+r_f)$
* $\underbrace{\frac{(1+r_f)-d}{u-d}}_{=p}$ $\underbrace{\frac{u-(1+r_f)}{u-d}}_{=(1-p)}$
Defining $p$ and $1-p$ as risk-neutral probabilities,
gives a simplified formula for the value of the call c
* $c = \frac{pc_u + (1-p)c_d}{1+r_f}$

**Two step:**

Substituting the values of c_u and c_d
* $c = [p^2c_{uu} + 2p(1-p)c_{ud} + (1-p)^2 pc_{dd}] \div (1+r_f)^2$

Where
* $c_{uu} = MAX[0,u^2S - X]$
* $c_{ud} = c_{du} = MAX[0,udS - X]$
* $c_{dd} = MAX[0,d^2S - X]$

For the numerical example given above we get
* $c = [p^2c_{uu} + 2p(1-p)c_{ud} + (1-p)^2 c_{dd}] \div (1+r_f)^2$

In [49]:
def calculate_option(S, X, u, d, rf, tp, ct):
    # Risk neutral probability
    p = (1 + rf - d) / (u - d)
    
    # Stock prices at different nodes
    Su = u * S
    Sd = d * S
    Suu = u * Su
    Sud = d * Su
    Sdd = d * Sd
    
    multiplier = 1 if tp == 'call' else -1
    
    # Option values at final nodes
    cuu = max(0, multiplier * (Suu - X))
    cud = max(0, multiplier * (Sud - X))
    cdd = max(0, multiplier * (Sdd - X))
    
    # Option values at intermediate nodes
    cu = (p * cuu + (1 - p) * cud) / (1 + rf)
    cd = (p * cud + (1 - p) * cdd) / (1 + rf)
    
    if (tp == 'call' and ct == 'european') or (tp == 'put' and ct == 'american'):
        cu = max(multiplier * (Su - X), cu)
        cd = max(multiplier * (Sd - X), cd)
    
    # Option value at initial node
    c0 = (p * cu + (1 - p) * cd) / (1 + rf)
    if tp == 'call':
        c0 = c0 if ct == 'american' else max(S - X, c0)
    else:
        c0 = max(X - S, c0) if ct == 'american' else max(0, c0)
        
    # Calculate m (option's delta)
    cu_0 = max(multiplier * (Su - X), 0)
    cd_0 = max(multiplier * (Sd - X), 0)
    p1 = S * (u - d)
    p2 = multiplier * (cu_0 - cd_0)
    m = p1 / p2 if tp == 'call' else p2 / p1
    
    return {
        'c0': c0,
        'cu': cu,
        'cd': cd,
        'cu_0': cu_0,
        'cd_0': cd_0,
        'cuu': cuu, 
        'cud': cud, 
        'cdd': cdd, 
        'p': p, 
        'm': m, 
    }

In [48]:
def plot_binomial_tree(result):
    # Create nodes (swapping x and y coordinates for horizontal orientation)
    nodes = [
        (0, 0, f"{result['c0']:.2f}"),
        (1, 1, f"{result['cu']:.2f}"), (1, -1, f"{result['cd']:.2f}"),
        (2, 2, f"{result['cuu']:.2f}"), (2, 0, f"{result['cud']:.2f}"), (2, -2, f"{result['cdd']:.2f}")
    ]

    # Create edges
    edges = [
        (0, 1), (0, 2), (1, 3), (1, 4), (2, 4), (2, 5)
    ]

    # Create figure
    fig = go.Figure()

    # Add nodes
    node_x, node_y, node_text = zip(*nodes)
    fig.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        marker=dict(size=30, color='skyblue'),
        text=node_text,
        textposition='middle center'
    ))

    # Add edges
    for edge in edges:
        fig.add_trace(go.Scatter(
            x=[nodes[edge[0]][0], nodes[edge[1]][0]],
            y=[nodes[edge[0]][1], nodes[edge[1]][1]],
            mode='lines',
            line=dict(color='grey', width=1)
        ))

    # Update layout
    fig.update_layout(
        title='Binomial Tree Option Pricing',
        showlegend=False,
        hovermode='closest',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis_range=[-2.5, 2.5],  # Adjust this to control vertical spread
        xaxis_range=[-0.5, 2.5]  # Adjust this to control horizontal spread
    )

    return fig

In [46]:
# Example usage
S = 50  # current stock price
X = 50  # exercise price
u = 1.2  # upwards movement
d = 0.6  # downwards movement
rf = 0.1  # risk-free rate
tp = 'put'  # option type 'call' or 'put'
ct = 'american'  # contract type 'american' or 'european'

result = calculate_option(S, X, u, d, rf, tp, ct)
fig = plot_binomial_tree(result)
fig.show()

print(f"Value of hedge ratio (m): {result['m']:.2f}")
print(
    f"One-period hedge portfolio payoffs: {result['m'] * S * u + result['cu_0']:.2f} or {result['m'] * S * d + result['cd_0']:.2f}")
print(f"Option Value in t=0 (P0): {result['c0']:.2f}")
print(f"Risk-Neutral Probability (p): {result['p']:.2f} and (p-1): {1 - result['p']:,.2f}")
print(f"Payoffs for 2-period {f'{ct.title()} {tp.title()}'} option:")
print(f"  - Up-Up: {result['cuu']:.2f}")
print(f"  - Up-Down: {result['cud']:.2f}")
print(f"  - Down-Down: {result['cdd']:.2f}")

Value of hedge ratio (m): 0.67
One-period hedge portfolio payoffs: 40.00 or 40.00
Option Value in t=0 (P0): 4.64
Risk-Neutral Probability (p): 0.83 and (p-1): 0.17
Payoffs for 2-period American Put option:
  - Up-Up: 0.00
  - Up-Down: 14.00
  - Down-Down: 32.00


## Black-Scholes Model

**Assumptions:**
* Stock price moves randomly in continuous time
    * Stock price modeled as geometric Brownian motion (return normally distributed, constant volatility)
* No dividends
* No market frictions: No arbitrage opportunities, no transaction costs, constant interest rate, no short-sale restrictions
 
**Variables:**
* $S$ = Current stock price
* $X$ = Exercise price
* $T$ = Time to expiration (in years)
* $r_f$ = Risk-free rate
* $\sigma$ = Firm asset volatility
* $\sigma^2$ = Variance of firm asset returns (per year)

$c = S N(d_1) - X e^{-r_f T} N(d_2)$

where 
* $d_1 = \frac{\ln(S/X)+r_fT}{\sigma\sqrt{T}} + \frac{1}{2}\sigma\sqrt{T}$ 
* $d_2 = d_1 - \sigma\sqrt{T}$

Note, BSM is equivalent to Binomial model, if a large number of steps is used and the following holds: $u = e^{\sigma\sqrt{\Delta T}}$

In [5]:
def black_scholes(S, X, rf, T, sig, tp):
    d1 = (math.log(S / X) + (rf * T)) / (sig * math.sqrt(T)) + 0.5 * sig * math.sqrt(T)
    d2 = d1 - sig * math.sqrt(T)
    
    multiplier = 1 if tp == 'call' else -1
    
    N1 = norm.cdf(multiplier * d1)
    N2 = norm.cdf(multiplier * d2)
    
    c0 = multiplier * (S * N1 - X * math.exp(-rf * T) * N2) # continuous time
    #c0 = multiplier * (S * N1 - (X / (1 + rf)**T) * N2) # discrete time
    
    neg = {1: '', -1: '-'}[multiplier]
    
    return {
        'd1': d1,
        'd2': d2,
        'N-d1': (f"N({neg}d1)", N1), 
        'N-d2': (f"N({neg}d2)", N2),
        'c0': c0
    }

In [6]:
#!TODO THIS DOESN'T WORK Currently with Tutorial 1 Question 3 someone please look into this


# S - Current stock price
S = 20
# X - Exercise price
X = 20
# T - Time to expiration (in years) calculated from:

# maturity in
days = 0
months = 6
years = 0

total_days = days + months * 365/12 + years * 365
T = total_days / 365

# rf - Risk-free rate
rf = 0.08
# variance - Variance of firm asset returns (per year)
variance = 0.36
# sig - Firm asset volatility
sig = None
tp = 'call' # option type 'call' or 'put'

if sig and variance:
    raise ValueError("Only one of sigma or variance should be provided")
elif variance:
    sig = math.sqrt(variance)
    
result = black_scholes(S, X, rf, T, sig, tp)

print(f"Value of European {tp.title()} option: {result['c0']:.2f}")
print('-' * 50)
print(f"d1: {result['d1']:.2f}, d2: {result['d2']:.2f}")
print(f"{result['N-d1'][0]}={result['N-d1'][1]:.4f}, {result['N-d2'][0]}={result['N-d2'][1]:.4f}")

Value of European Call option: 3.70
--------------------------------------------------
d1: 0.31, d2: -0.12
N(d1)=0.6204, N(d2)=0.4531


# Merton Model
Equity is call option on market value of firm with debt value as strike

**Setting:**
* Firm with firm value V
    * Consisting of risky equity S and debt
* Debt is zero coupon bond
    * With face value D and maturity in T years from now
* Debt is secured by assets of firm
* Firm pays no dividends

Value of equity S at maturity T is given by
* $S = max[0, V - D]$
* At maturity T, equity holders get value of the firm V in excess of debt value D
* If V < D, the firm will default

$S = V \cdot N(d_1) - D \cdot e^{-r_f T} \cdot N(d_2)$

where:
* $d_1 = \frac{\ln(V/D) + r_f T}{\sigma \sqrt{T}} + \frac{1}{2} \sigma \sqrt{T}$
* $d_2 = d_1 - \sigma \sqrt{T}$

For calculating a firm's value as a call option (YTM = Yield to Maturity):
* $YTM = \sqrt[T]{\frac{D}{B}} - 1$

where:
* $D$ = Zero-Bond
* $B = V - S$: Value of debt

For calculating the Recovery Rate in the risk-neutral world:
* $RR_n = \frac{B \cdot e^{r_f T} - D \cdot N(d_2)}{D \cdot (1 - N(d_2))}$

In [7]:
def merton(V, D, T, rf, sig):
    '''
    Calculates the firma's equity as a call option
    Parameters
    ----------
    V: float
    firma's value
    D: float
    zero-bond
    T: float
    time to maturity (in years)
    rf: float
    risk-free rate
    sig: float
    firma's asset volatility
    '''
    d1 = (math.log(V / D) + rf * T) / (sig * math.sqrt(T)) + 0.5 * sig * math.sqrt(T)
    d2 = d1 - sig * math.sqrt(T)
    d1 = round(d1, 2)
    d2 = round(d2, 2)
    N1 = norm.cdf(d1)
    N1 = round(N1, 4)
    N2 = norm.cdf(d2)
    N2 = round(N2, 4)
    S = V * N1 - D * math.exp(-rf * T) * N2
    
    B = V - S
    
    RRN = (B * math.exp(rf * T) - D * N2) / (D * (1 - N2))
    ytm = (D / B) ** (1 / T) - 1
    
    return {
        'd1': d1,
        'd2': d2,
        'N-d1': N1,
        'N-d2': N2,
        'S': S,
        'B': B,
        'YTM': ytm,
        'RRN': RRN
    }

In [8]:
V = 5_000_000
D = 4_000_000
# Zero-bond
# Time to maturity
days = 0
months = 0
years = 10

total_days = days + months * 365/12 + years * 365
T = total_days / 365
# Risk-free rate
rf = 0.05

# Firm asset volatility
variance = 0.5
sig = None

if sig and variance:
    raise ValueError("Only one of sigma or variance should be provided")
elif variance:
    sig = math.sqrt(variance)

result = merton(V, D, T, rf, sig)

print(f"Value of firm's equity: {result['S']:.2f}")
print(f"Value of firm's debt: {result['B']:.2f}")
print('-' * 50)
print(f"The YTM (Yield to Maturity) is: {result['YTM']:.4f}")
print(f"The Recovery Rate in the risk-neutral world is: {result['RRN']:.4f}")
print('-' * 50)
print(f"d1: {result['d1']:.2f}, d2: {result['d2']:.2f}")
print(f"N(d1)={result['N-d1']:.4f}, N(d2)={result['N-d2']:.4f}")






Value of firm's equity: 4104368.86
Value of firm's debt: 895631.14
--------------------------------------------------
The YTM (Yield to Maturity) is: 0.1614
The Recovery Rate in the risk-neutral world is: 0.1966
--------------------------------------------------
d1: 1.44, d2: -0.79
N(d1)=0.9251, N(d2)=0.2148


# Put-Call Parity and Arbitrage

Put-call parity is a fundamental principle in options pricing that establishes a relationship between European put options, call options, and their underlying asset.

## Key Concepts:

1. **Equation**: S + P = C + PV(X)
   - S: Current stock price
   - P: Put option price
   - C: Call option price
   - PV(X): Present value of strike price

2. **Portfolio Equivalence**: 
   - Long stock + Long put = Long call + Cash (equal to PV of strike price)

3. **Arbitrage-Free Condition**: 
   - Ensures no risk-free profit opportunities exist

4. **Applications**:
   - Validating option pricing models
   - Identifying mispriced options
   - Deriving implied volatility

5. **Assumptions**:
   - European-style options
   - No dividends
   - Efficient markets
   - No transaction costs

In [9]:
def put_call_parity(c0, p0, S0, X, T, rf, tp):
    if tp == 'discrete':
        P0 = c0 - S0 + X / ((1 + rf) ** T)
        C0 = p0 + S0 - X / ((1 + rf) ** T)
    else:
        P0 = c0 - S0 + X * math.exp(-rf * T)
        C0 = p0 + S0 - X * math.exp(-rf * T)

    return {
        'P0': P0, # Put option value
        'C0': C0  # Call option value
    }

In [10]:
# c0 - Call option value
c0 = 3.7
# p0 - Put option value
p0 = 0
# S0 - Current stock price
S0 = 20
# X - Exercise price
X = 20
# T - Time to maturity (in years)
T = 0.5
# rf - Risk-free rate
rf = 0.08
# tp - Type of parity
# tp = 'discrete'
tp = 'continuous'
result = put_call_parity(c0, p0, S0, X, T, rf, tp)


print(f"Value of Put Option (P0): {result['P0']:.2f}")
print(f"Value of Call Option (C0): {result['C0']:.2f}")

Value of Put Option (P0): 2.92
Value of Call Option (C0): 0.78


# Net Present Value (NPV)

$NPV = -I_0 + \sum_{t=1}^{n} \frac{E(FCF_t)}{(1 + WACC)^t} > 0$

where:
* $I_0$: Initial investment
* $FCF_t$: Free cash flow at time $t$
* $WACC$: Weighted average cost of capital
* $n$: Number of periods

**Advantages**:
* Simple to implement

**Disadvantages**:
* No flexibility after investment decision
* Underestimates the value of a project

**Implicit assumption: Pre-commitment to a deterministic course of action**
* NPV-method not suitable for flexible, multi-period decision making under uncertainty
* Real option explicitly model this flexibility

In [11]:
def calculate_NPV_no_expansion(I, sig, V, T, n):
    '''
    Calculates the NPV and node values for a binomial tree without expansion
    Parameters
    ----------
    I: float
        initial investment
    sig: float
        volatility
    V: float
        present value w/o flexibility
    T: float
        time to maturity (in years)
    n: int
        steps in the tree
    
    Returns
    -------
    dict
        A dictionary containing all calculated values
    '''
    npv = V - I
    u = math.exp(sig * math.sqrt(T / n))
    u = round(u, 2)
    d = 1 / u
    d = round(d, 2)
    uV = u * V
    uV = round(uV, 2)
    dV = d * V
    dV = round(dV, 2)
    udV = uV * d
    udV = round(udV, 2)
    uuV = uV * u
    uuV = round(uuV, 2)
    ddV = dV * d

    return {
        'npv': npv,
        'u': u,
        'd': d, 
        'V': V, 
        'uV':uV, 
        'dV':dV, 
        'uuV':uuV, 
        'udV':udV, 
        'ddV':ddV,
    }

In [12]:
def plot_binomial_tree(results):
    '''
    Plots the binomial tree using Plotly
    Parameters
    ----------
    results: dict
        Dictionary containing the calculated values from calculate_NPV_no_expansion
    '''
    # Create nodes for the tree
    nodes = [
        (0, 0, f"{results['V']:.2f}"),
        (1, 1, f"{results['uV']:.2f}"), (1, -1, f"{results['dV']:.2f}"),
        (2, 2, f"{results['uuV']:.2f}"), (2, 0, f"{results['udV']:.2f}"), (2, -2, f"{results['ddV']:.2f}")
    ]

    # Create edges
    edges = [
        (0, 1), (0, 2), (1, 3), (1, 4), (2, 4), (2, 5)
    ]

    # Create figure
    fig = go.Figure()

    # Add nodes
    node_x, node_y, node_text = zip(*nodes)
    fig.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        marker=dict(size=30, color='skyblue'),
        text=node_text,
        textposition='middle center'
    ))

    # Add edges
    for edge in edges:
        fig.add_trace(go.Scatter(
            x=[nodes[edge[0]][0], nodes[edge[1]][0]],
            y=[nodes[edge[0]][1], nodes[edge[1]][1]],
            mode='lines',
            line=dict(color='grey', width=1)
        ))

    # Update layout
    fig.update_layout(
        title='Binomial Tree - NPV No Expansion',
        showlegend=False,
        hovermode='closest',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-0.5, 2.5]),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-2.5, 2.5]),
        annotations=[
            dict(x=1, y=2.5, xref="x", yref="y", text=f"u={results['u']:.2f}", showarrow=False),
            dict(x=1, y=-2.5, xref="x", yref="y", text=f"d={results['d']:.2f}", showarrow=False)
        ]
    )

    # Show the plot
    fig.show()


In [13]:
# Example usage
I = 20
sig = 0.15
V = 30
T = 2
n = 2

# Calculate the NPV and tree values
results = calculate_NPV_no_expansion(I, sig, V, T, n)

# Print the NPV
print(f"The NPV without expansion is: {results['npv']:.2f}")

# Plot the binomial tree
plot_binomial_tree(results)

The NPV without expansion is: 10.00


# Real Options Analysis (ROA)

Types of Real Options:
* **Expansion option**: Growth option on an underlying asset that assumes precommitment of
a series of investments to growing demand over time
    * American call with 'cost of expandable investment' as exercise price and 'multiple of
the value of the underlying risky asset' as option value
* **Contraction option**: Option to receive cash for partially giving up the use of asset
    * American put with present value of cash as exercise price and fraction of the value of
operations given up as value of the underlying
* **Abandonment option**: Right to sell an asset for given price, which can change through
time rather than continuing to hold it (American put)
* **Extension option**: Allows manager to pay a cost for the ability to extend the life of a
project (European call with cost of extension as exercise price)
* **Deferral option**: Right to postpone the start of a project (American call)
* **Switching option**: Right to turn a project on and off
* **Compound options**: Options on options

**Advantages:**
* Incorporates flexibility
* Arbitrage-free valuation
* Valuation is not based on physical probabilities

In [14]:
def calculate_ROA(sig, V, T, n, rf, A, i):
    '''
    Calculates the Real Options Analysis (ROA) decision tree
    Parameters
    ----------
    sig: float
        volatility
    V: float
        present value w/o flexibility
    T: float
        time to maturity (in years)
    n: int
        steps in the tree
    rf: float
        risk-free rate
    A: float
        additional investment
    i: float
        increase in value caused by investment A
    
    Returns
    -------
    dict
        A dictionary containing all calculated values and decisions
    '''
    u = round(math.exp(sig * math.sqrt(T / n)), 2)
    d = round(1 / u, 2)

    uV = round(u * V, 2)
    dV = round(d * V, 2)
    uuV = round(u * uV, 2)
    udV = round(u * dV, 2)
    ddV = round(d * dV, 2)

    p = round((1 + rf - d) / (u - d), 2)

    ROAuu = round(max(uuV, i * uuV - A), 2)
    print(udV)
    ROAud = round(max(udV, i * udV - A), 2)
    ROAdd = round(max(ddV, i * ddV - A), 2)

    uucmd = "don't expand" if ROAuu > i * uuV - A else "expand"
    udcmd = "don't expand" if ROAud > i * udV - A else "expand"
    ddcmd = "don't expand" if ROAdd > i * ddV - A else "expand"

    ROAu = round(max((p * ROAuu + (1 - p) * ROAud) / (1 + rf), i * uV - A), 2)
    ROAd = round(max((p * ROAud + (1 - p) * ROAdd) / (1 + rf), i * dV - A), 2)

    ucmd = "don't expand" if ROAu > i * uV - A else "expand"
    dcmd = "don't expand" if ROAd > i * dV - A else "expand"

    ROA = round((p * ROAu + (1 - p) * ROAd) / (1 + rf), 2)

    return {
        'ROA': round(ROA, 2),
        'u': round(u, 2), 'd': round(d, 2), 'p': round(p, 2),
        'uV': uV, 'dV': dV, 'uuV': uuV, 'udV': udV, 'ddV': ddV,
        'ROAu': ROAu, 'ROAd': ROAd, 'ROAuu': ROAuu, 'ROAud': ROAud, 'ROAdd': ROAdd,
        'ucmd': ucmd, 'dcmd': dcmd, 'uucmd': uucmd, 'udcmd': udcmd, 'ddcmd': ddcmd
    }

In [15]:
def plot_ROA_tree(results):
    '''
    Plots the ROA decision tree using Plotly
    Parameters
    ----------
    results: dict
        Dictionary containing the calculated values from calculate_ROA
    '''
    # Create nodes for the tree
    nodes = [
        (0, 0, f"{results['ROA']:.2f}"),
        (1, 1, f"{results['ROAu']:.2f}\n{results['ucmd']}"),
        (1, -1, f"{results['ROAd']:.2f}\n{results['dcmd']}"),
        (2, 2, f"{results['ROAuu']:.2f}\n{results['uucmd']}"),
        (2, 0, f"{results['ROAud']:.2f}\n{results['udcmd']}"),
        (2, -2, f"{results['ROAdd']:.2f}\n{results['ddcmd']}")
    ]

    # Create edges
    edges = [
        (0, 1), (0, 2), (1, 3), (1, 4), (2, 4), (2, 5)
    ]

    # Create figure
    fig = go.Figure()

    # Add nodes
    node_x, node_y, node_text = zip(*nodes)
    fig.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        marker=dict(size=40, color='lightblue'),
        text=node_text,
        textposition='middle center'
    ))

    # Add edges
    for edge in edges:
        fig.add_trace(go.Scatter(
            x=[nodes[edge[0]][0], nodes[edge[1]][0]],
            y=[nodes[edge[0]][1], nodes[edge[1]][1]],
            mode='lines',
            line=dict(color='grey', width=1)
        ))

    # Update layout
    fig.update_layout(
        title='ROA Decision Tree',
        showlegend=False,
        hovermode='closest',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-0.5, 2.5]),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-2.5, 2.5]),
        annotations=[
            dict(x=1, y=2.5, xref="x", yref="y", text=f"u={results['u']:.4f}, p={results['p']:.4f}", showarrow=False),
            dict(x=1, y=-2.5, xref="x", yref="y", text=f"d={results['d']:.4f}, 1-p={1 - results['p']:.4f}",
                 showarrow=False)
        ]
    )

    # Show the plot
    fig.show()

In [16]:
# Example usage
# I - Initial investment
I = 20
# sig - volatility
sig = 0.15
# V - present value w/o flexibility
V = 30
# T - time to maturity (in years)
T = 2
# n - steps in the tree
n = 2
# rf - risk-free rate
rf = 0.05
# A - additional investment
A = 5
# i - increase in value caused by investment A (20% increase -> 1.2)
i = 1.2

# Calculate ROA
results = calculate_ROA(sig, V, T, n, rf, A, i)

# Print results
print(f"ROA = {results['ROA']:.2f}")
print(f"u = {results['u']:.2f}, d = {results['d']:.2f}")
print(f"p = {results['p']:.2f}")
print(f"ROA NPV: {results['ROA'] - I:.2f}")

# Plot the ROA decision tree
plot_ROA_tree(results)

29.93
ROA = 31.47
u = 1.16, d = 0.86
p = 0.63
ROA NPV: 11.47


## Decision Tree Analysis (DTA)

Note that the following relationship must hold on an efficient market for the asset values of any firm:
* $V_0 = \frac{qV_q + (1 - q)V_d}{1 + WACC}$

The relationship can be re-written as
* $q = \frac{1 + WACC - d}{u -d}$
or
* $1 + WACC = q * u + (1 - q) * d$

**Advantages**:
* Incorporates flexibility

**Disadvantages**:
* Does not obey to the law of one price
* A constant WACC is assumed
* Physical probabilities are used

In [17]:
def calc_p(rf, u, d):
    return (1 + rf - d) / (u - d)

def calc_adjusted_rf(q, u, d):
    return q * (u - d) + d - 1

def multi_period_PV(uI, dI, uO, dO, In, Out, rf, periods, q=None, risk_type='neutral', flexibility=False):
    def calculate_cashflow(amount, up_factor, down_factor, period):
        return [amount * (up_factor ** (period - i)) * (down_factor ** i) for i in range(period + 1)]

    def calculate_probability(p, period):
        return [math.comb(period, i) * (p ** (period - i)) * ((1 - p) ** i) for i in range(period + 1)]

    pIn = calc_p(rf, uI, dI)
    pOut = calc_p(rf, uO, dO)
    rf_In = rf
    rf_Out = rf

    if risk_type == 'adjusted':
        if q is None:
            raise ValueError("q must be provided for risk-adjusted calculations")
        rf_In = calc_adjusted_rf(q, uI, dI)
        rf_Out = calc_adjusted_rf(q, uO, dO)
        pIn = pOut = q

    PV_In = 0
    PV_Out = 0

    for period in range(periods + 1):
        cashflows_in = calculate_cashflow(In, uI, dI, period)
        cashflows_out = calculate_cashflow(Out, uO, dO, period)
        probabilities_in = calculate_probability(pIn, period)
        probabilities_out = calculate_probability(pOut, period)

        if flexibility:
            net_cashflows = [out - inp for out, inp in zip(cashflows_out, cashflows_in)]
            PV_this_period = sum(max(0, cf) * prob / (1 + rf_Out) ** period 
                                 for cf, prob in zip(net_cashflows, probabilities_out))
            PV_Out += PV_this_period
        else:
            PV_In += sum(cf * prob / (1 + rf_In) ** period for cf, prob in zip(cashflows_in, probabilities_in))
            PV_Out += sum(cf * prob / (1 + rf_Out) ** period for cf, prob in zip(cashflows_out, probabilities_out))

    return PV_Out - PV_In if not flexibility else PV_Out

In [18]:
uI = 1.5
dI = 0.8
uO = 1.8
dO = 0.6
In = 85
Out = 100
rf = 0.08
periods=2

result = multi_period_PV(uI, dI, uO, dO, In, Out, rf, periods)
print(f"PV difference over {periods} periods: {result}")

# For risk-adjusted calculation
q = 0.5
result_adjusted = multi_period_PV(uI, dI, uO, dO, In, Out, rf, periods, q=q, risk_type='adjusted')
print(f"Risk-adjusted PV difference over {periods} periods: {result_adjusted}")

# With flexibility
result_flexible = multi_period_PV(uI, dI, uO, dO, In, Out, rf, periods, flexibility=True)
print(f"PV difference with flexibility over {periods} periods: {result_flexible}")

PV difference over 2 periods: 45.0
Risk-adjusted PV difference over 2 periods: 45.0
PV difference with flexibility over 2 periods: 55.123456790123456


In [19]:
def calculate_DTA_no_flex(V0, sig, T, n, WACC):
    '''
    Calculates the DTA tree without flexibility
    Parameters
    ----------
    V0: float
        present fair-value of the project
    sig: float
        volatility
    T: float
        time (in years)
    n: int
        steps in the tree
    WACC: float
        risk-neutral rate
    
    Returns
    -------
    dict
        A dictionary containing all calculated values
    '''
    u = round(math.exp(sig * math.sqrt(T / n)), 4)
    d = round(1 / u, 4)

    uV = round(u * V0, 2)
    dV = round(d * V0, 2)
    uuV = round(u * uV, 2)
    udV = round(u * dV, 2)
    ddV = round(d * dV, 2)

    q = round((1 + WACC - d) / (u - d), 4)

    return {
        'V0': round(V0, 2),
        'u': round(u, 4), 'd': round(d, 4), 'q': round(q, 4),
        'uV': round(uV), 'dV': round(dV, 2),
        'uuV': round(uuV, 2), 'udV': round(udV, 2), 'ddV': round(ddV, 2)
    }

In [20]:
def plot_DTA_no_flex(results):
    '''
    Plots the DTA tree without flexibility using Plotly
    Parameters
    ----------
    results: dict
        Dictionary containing the calculated values from calculate_DTA_no_flex
    '''
    # Create nodes for the tree
    nodes = [
        (0, 0, f"{results['V0']:.2f}"),
        (1, 1, f"{results['uV']:.2f}"),
        (1, -1, f"{results['dV']:.2f}"),
        (2, 2, f"{results['uuV']:.2f}"),
        (2, 0, f"{results['udV']:.2f}"),
        (2, -2, f"{results['ddV']:.2f}")
    ]

    # Create edges
    edges = [
        (0, 1), (0, 2), (1, 3), (1, 4), (2, 4), (2, 5)
    ]

    # Create figure
    fig = go.Figure()

    # Add nodes
    node_x, node_y, node_text = zip(*nodes)
    fig.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        marker=dict(size=40, color='lightgreen'),
        text=node_text,
        textposition='middle center'
    ))

    # Add edges
    for edge in edges:
        fig.add_trace(go.Scatter(
            x=[nodes[edge[0]][0], nodes[edge[1]][0]],
            y=[nodes[edge[0]][1], nodes[edge[1]][1]],
            mode='lines',
            line=dict(color='grey', width=1)
        ))

    # Update layout
    fig.update_layout(
        title='DTA Tree without Flexibility',
        showlegend=False,
        hovermode='closest',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-0.5, 2.5]),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-2.5, 2.5]),
        annotations=[
            dict(x=1, y=2.5, xref="x", yref="y", text=f"u={results['u']:.4f}, q={results['q']:.4f}", showarrow=False),
            dict(x=1, y=-2.5, xref="x", yref="y", text=f"d={results['d']:.4f}, 1-q={1 - results['q']:.4f}",
                 showarrow=False)
        ]
    )

    # Show the plot
    fig.show()

In [21]:
# Example usage
WACC = 0.12
i = 80  # investment
T = 2  # years
n = 2  # steps in the tree
V0 = 100  # initial projected value of the project
sig = 0.4  # volatility
rf = 0.05  # risk-free

# Calculate DTA without flexibility
results = calculate_DTA_no_flex(V0, sig, T, n, WACC)

# Print results
print(f'NPV = {results["V0"] - i:.2f}')
print('-' * 50)
print(f'The objective probabilities are:')
print(f'q = {results["q"]:.4f} and 1-q = {1 - results["q"]:.4f}')
print('-' * 50)
print(f'u = {results["u"]:.4f} and d = {results["d"]:.4f}')

# Plot the DTA tree without flexibility
plot_DTA_no_flex(results)

NPV = 20.00
--------------------------------------------------
The objective probabilities are:
q = 0.5474 and 1-q = 0.4526
--------------------------------------------------
u = 1.4918 and d = 0.6703


In [22]:
def realize_option(V, DTA, expansion, price):
    return max(DTA, expansion * V - price), "expand" if DTA < expansion * V - price else "don't expand"

def expansion_threshold(expansion, price):
    return price / (expansion - 1)

def calculate_next_step(Vu, Vd, WACC, q):
    return (q * Vu + (1 - q) * Vd) / (1 + WACC)

def calculate_DTA_flex(V0, sig, T, n, WACC, expansion, price):
    '''
    Calculates the DTA tree with flexibility
    Parameters
    ----------
    V0: float
        present fair-value of the project
    sig: float
        volatility
    T: float
        time (in years)
    n: int
        steps in the tree
    WACC: float
        risk-neutral rate
    expansion: float
        expansion rate (if contraction: <1)
    price: float
        price of expansion (if contraction: <0)
    
    Returns
    -------
    dict
        A dictionary containing all calculated values and decisions
    '''
    u = round(math.exp(sig * math.sqrt(T / n)), 4)
    d = round(1 / u, 4)

    # Calculate stock prices at different nodes
    uV = round(u * V0, 2)
    dV = round(d * V0, 2)
    uuV = round(u * uV, 2)
    udV = round(u * dV, 2)
    ddV = round(d * dV, 2)

    q = round((1 + WACC - d) / (u - d), 4)

    # Calculate DTA at final nodes
    uuDTA, uucmd = realize_option(uuV, uuV, expansion, price)
    udDTA, udcmd = realize_option(udV, udV, expansion, price)
    ddDTA, ddcmd = realize_option(ddV, ddV, expansion, price)

    # Calculate DTA at intermediate nodes
    uDTA_next = calculate_next_step(uuDTA, udDTA, WACC, q)
    dDTA_next = calculate_next_step(udDTA, ddDTA, WACC, q)

    uDTA, ucmd = realize_option(uV, uDTA_next, expansion, price)
    dDTA, dcmd = realize_option(dV, dDTA_next, expansion, price)

    # Calculate overall DTA
    DTA = calculate_next_step(uDTA, dDTA, WACC, q)

    return {
        'V0': V0, 'DTA': round(DTA, 2),
        'u': u, 'd': d, 'q': q,
        'uV': uV, 'dV': dV,
        'uuV': uuV, 'udV': udV, 'ddV': ddV,
        'uDTA': round(uDTA, 2), 'dDTA': round(dDTA, 2),
        'uuDTA': round(uuDTA, 2), 'udDTA': round(udDTA, 2), 'ddDTA': round(ddDTA, 2),
        'ucmd': ucmd, 'dcmd': dcmd,
        'uucmd': uucmd, 'udcmd': udcmd, 'ddcmd': ddcmd
    }

In [23]:
def plot_DTA_flex(results):
    '''
    Plots the DTA tree with flexibility using Plotly
    Parameters
    ----------
    results: dict
        Dictionary containing the calculated values from calculate_DTA_flex
    '''
    # Create nodes for the tree
    nodes = [
        (0, 0, f"{results['DTA']:.2f}"),
        (1, 1, f"{results['uDTA']:.2f}\n{results['ucmd']}"),
        (1, -1, f"{results['dDTA']:.2f}\n{results['dcmd']}"),
        (2, 2, f"{results['uuDTA']:.2f}\n{results['uucmd']}"),
        (2, 0, f"{results['udDTA']:.2f}\n{results['udcmd']}"),
        (2, -2, f"{results['ddDTA']:.2f}\n{results['ddcmd']}")
    ]

    # Create edges
    edges = [
        (0, 1), (0, 2), (1, 3), (1, 4), (2, 4), (2, 5)
    ]

    # Create figure
    fig = go.Figure()

    # Add nodes
    node_x, node_y, node_text = zip(*nodes)
    fig.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        marker=dict(size=40, color='lightyellow'),
        text=node_text,
        textposition='middle center'
    ))

    # Add edges
    for edge in edges:
        fig.add_trace(go.Scatter(
            x=[nodes[edge[0]][0], nodes[edge[1]][0]],
            y=[nodes[edge[0]][1], nodes[edge[1]][1]],
            mode='lines',
            line=dict(color='grey', width=1)
        ))

    # Update layout
    fig.update_layout(
        title='DTA Tree with Flexibility',
        showlegend=False,
        hovermode='closest',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-0.5, 2.5]),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=[-2.5, 2.5]),
        annotations=[
            dict(x=1, y=2.5, xref="x", yref="y", text=f"u={results['u']:.4f}, q={results['q']:.4f}", showarrow=False),
            dict(x=1, y=-2.5, xref="x", yref="y", text=f"d={results['d']:.4f}, 1-q={1 - results['q']:.4f}",
                 showarrow=False)
        ]
    )

    # Show the plot
    fig.show()

In [24]:
# Example usage
WACC = 0.12
i = 80  # investment
T = 2  # years
n = 2  # steps in the tree
V0 = 100  # initial projected value of the project
sig = 0.2  # volatility
rf = 0.05  # risk-free
expansion = 0.6
price = -42

# Calculate DTA with flexibility
results = calculate_DTA_flex(V0, sig, T, n, WACC, expansion, price)

print(f'The DTA NPV with flexibility is: {results["DTA"] - i:.2f}')
print('-' * 50)
print(f'The objective probabilities are:')
print(f'q = {results["q"]:.4f} and 1-q = {1 - results["q"]:.4f}')
print(f'Expansion threshold: {expansion_threshold(expansion, price):.2f}')
print(f'The Option Value is: {results["DTA"] - results["V0"]:.2f}')
print('-' * 50)
print(f'u = {results["u"]:.4f} and d = {results["d"]:.4f}')


      

# Plot the DTA tree with flexibility
plot_DTA_flex(results)

The DTA NPV with flexibility is: 22.38
--------------------------------------------------
The objective probabilities are:
q = 0.7482 and 1-q = 0.2518
Expansion threshold: 105.00
The Option Value is: 2.38
--------------------------------------------------
u = 1.2214 and d = 0.8187


## DTA x ROA

**Difference to decision tree is branch-dependent discount rate**

* Note, decision tree approach (DTA) adjusts discount rate for risk (by using the WACC) and uses physical probabilities
* Real options approach (ROA) discounts with risk-free discount rate and adjusts probabilities for risk by using the risk-neutral probability
* Note, call value formula can be modified to

$C = \frac{pC_u}{1+r_f} + \frac{(1-p)C_d}{1+r_f} = \frac{qC_u}{(1+r_f)\frac{q}{p}} + \frac{(1-q)C_d}{(1+r_f)\frac{(1-q)}{(1-p)}} = \frac{qC_u}{(1+r_u)} + \frac{(1-q)C_d}{(1+r_d)}$

* Decision tree method uses single discount rate for both branches real option approach implicitly uses branch-dependent discount rates ($r_u$ and $r_d$)

**How to choose?**
1) In principle ROA is always the better choice as it is the only consistent (arbitrage free)
approach to value flexibility.
2) Problem: there must be an observable market price for the project without flexibility. This is
often not the case (think about innovations, which are not yet traded by definition).
3) In such case the only solution is to use DTA, even though one has to be aware of all the
problems associated with this method. But most likely it is better to accept this problem
rather to ignore flexibility at all.
4) In practice, must approaches labelled as real options, in reality, are a DTA.

Call value formula: 

$C = \frac{pC_u}{1 + r_f} + \frac{(1 - p)C_d}{1 + r_f} = \frac{qC_u}{1 + r_f} + \frac{(1 - q)C_d}{1 + ru} + \frac{(1 - q)C_d}{1 + rd}$

# Staging Investments

* Companies often have to stage investment tranches, because of technological or capacity reasons. This is a typical problem in R&D investments.
* By doing so real options are realized (deferral, abandonment, expansion, etc.).
* Of course, there is also a downside, for instance because there is a loss in the time value of money of future cash flows. Moreover, it could also be that future investments become more expensive or that a competitive advantage is lost.
* However, staging give additional flexibility leading to a potential additional value.
* The company has to find a way to maximize this value.

General Idea:
* it's not a good idea to start with the most expensive stage.
* It's better to start with the most risky (least successful) investment stage, as the outcome of
this stage is most informative regarding the overall viability of the project.
* starting with the most lengthy project tends to be an advantage as the PV of the
investments for the succeeding stages is smaller (provided that there is no cost of
postponing, which is not so clear, for instance because of inflation).
* it seems to be beneficial to invest in less capital intensive, riskier and lengthier projects first.
* Typically it will be impossible to monotonically order the projects according to the three criteria
mentioned before. 
* Therefore, we are looking for an ordering taking all three dimensions into
account. This is fulfilled by the following failure cost criteria:

$\frac{1 - PV(success)}{PV(investment)}$

* $PV(success)$: is the expected marginal present value contribution of 1 dollar revenue
(which will only be generated, if the overall project is successful).
* $(PV investment)$: is the present value of the necessary investment for the specific stage.
* Note that this rule is similar to the profitability index, which was defined as
NPV/Investment.

The approach can be used in a ROA or DTA setting.
* $PV(investment) =$ cost of project
* $PV(success) = \frac{Prob(Project)}{(1 + WACC)^T}$


In [25]:
def calculate_final_npv(rf_yields, cost, profit, sucess_p, yields_p=None):
    '''
    Calculates the NPV at the final step
    Parameters
    ----------
    rf_yields: list[float]
    yield of the risk-free rate, list of possible yields
    cost: float
    cost at the final step of project (factory cost)
    profit: float
    profit per year after the project is done and is sucessfull
    sucess_p: float
    probability of sucess of project
    yields_p: list[float]
    probability of each yield
    '''
    return sucess_p * np.average(np.maximum(0, profit / np.array(rf_yields) - cost), weights=yields_p)


def calculate_initial_npv(yearly_cost, rf, final_npv, t):
    '''
    Calculates the NPV at t=0
    Parameters
    ----------
    yearly_cost: float
    cost (per year) of R and D
    rf: float
    risk-free rate
    final_npv: float
    NPV at final time step, calculated with ```calcute_final_npv()```
    t: int
    time steps
    '''
    NPV = 0
    for i in range(t):
        NPV -= yearly_cost / (1 + rf) ** i
    NPV += final_npv / (1 + rf) ** (t)

    return NPV

In [26]:
rf_yields = [.12, .1, .08, .05]
cost = 1000
profit = 100
sucess_p = .25
final_npv = calculate_final_npv(rf_yields, cost, profit, sucess_p)
yearly_cost = 10
rf = .1
t = 5
NPV = calculate_initial_npv(yearly_cost, rf, final_npv, t)

print(f'There is a {sucess_p * 100}% chance of success so that the expected value at time {t} is: {final_npv:.4f}')
print(f'The NPV of the development opportunity at time 0 is therefore: {NPV:.4f}')


There is a 25.0% chance of success so that the expected value at time 5 is: 78.1250
The NPV of the development opportunity at time 0 is therefore: 6.8108
